In [1]:
from tortus import Tortus
import pandas as pd
import os

HTML(value="<h1 style='text-align:center'>t &nbsp; <span style=        'color:#36a849'>o</span> &nbsp; r &nbsp…

HTML(value="<h3 style='text-align:center'>        easy text annotation in a Jupyter Notebook</h3>")

In [2]:
def annotate_tweets(path, text_column = "annotate_text", num_records=100, prev_annotations = None, additional_labels = []):
    df = pd.read_csv(path)
    output_path = create_path(path, "second_annotation", True)
    
    if os.path.exists(output_path):
        print(f"Annotations already exist for: {output_path}, adding to these annotations")
        prev_annotations = pd.read_csv(output_path, index_col = "Unnamed: 0")
        
    tortus = Tortus(df, "annotate_text", num_records=num_records, annotations=prev_annotations, labels=["full_standard_english", "not-syntactic_standard_english", "non_standard_english", "code-switched", "some_english", "not_english"] + additional_labels, id_column = "id_column")
    tortus.annotate()
    return tortus

def create_path(path, new_first_dir_name, makedirs = False):
    temp_path_list = os.path.dirname(path).split('/')
    temp_path_list[0] = new_first_dir_name
    outdirs = '/'.join(temp_path_list)
    basename = os.path.basename(path)
    if makedirs:
        os.makedirs(outdirs, exist_ok = True)
    return os.path.join(outdirs, basename)

def merge_and_save_annotations(tortus, first_annotation_path):
    
    first_annotation = pd.read_csv(first_annotation_path, index_col = "Unnamed: 0")
    first_annotation = first_annotation.rename(columns = {'label': 'label_1', 'annotated_at': 'annotated_at_1'})
    
    second_annotation = tortus.annotations
    second_annotation.to_csv(create_path(first_annotation_path, "second_annotation", True))
    second_annotation = second_annotation.rename(columns = {'label': 'label_2', 'annotated_at': 'annotated_at_2'})

    original = pd.read_csv(create_path(first_annotation_path, "data"), index_col = "Unnamed: 0")
    
    merged = first_annotation.merge(original, how = "left", left_on = "id_column", right_on = "id")
    merged = merged.merge(second_annotation, how = "left", on = "id_column")
    merged = merged[["id_column", "rawContent","cleaned_content", "label_1", "label_2", "english_relative_frequency", "date", "annotated_at_1", "annotated_at_2"]]
    merged.to_csv(create_path(first_annotation_path, "complete_annotation", True))
    
    return merged

In [3]:
first_annotation_path = "first_annotation/Singapore/tweets_over_period/24400_tweets_over_period/0.7_to_0.8_english_words.csv"
tortus = annotate_tweets(first_annotation_path, num_records = 10)

HTML(value='Click on the label corresponding with the text below. Each selection requires                 conf…

HTML(value='<h4><b>Raw:</b> Mas maganda pa din yung CLASSY  at Quality tweets nababasa under our tags kesa HAT…

GridBox(children=(VBox(children=(Box(children=(Button(description='full_standard_english', layout=Layout(borde…

Output()

In [109]:
output_df = merge_and_save_annotations(tortus, first_annotation_path)

,id_column,rawContent,cleaned_content,label_1,label_2,english_relative_frequency,date,annotated_at_1,annotated_at_2
0,22605,@9NewsSyd @cokeefe9 Ooooh another Qantas PR “l...,ooooh another qantas pr look over there moment...,full_standard_english,full_standard_english,0.785714,2022-08-16 23:52:19+00:00,2022-11-17 15:07:11,2022-11-17 15:08:04
1,9954,"I'm at Oasis Primary School in Damai, Punggol ...",im at oasis primary school in damai punggol,full_standard_english,full_standard_english,0.750000,2022-04-11 23:15:34+00:00,2022-11-17 15:07:13,2022-11-17 15:08:05
2,2576,@YouTube @marionskitchen 's XO sauce recipe,s xo sauce recipe,full_standard_english,full_standard_english,0.750000,2022-01-27 22:18:21+00:00,2022-11-17 15:07:14,2022-11-17 15:08:02
